<a href="https://colab.research.google.com/github/senudidinaya/Smart-Agri-Suite/blob/main/AI/Cultivatior%20Intent%20Module/Gate1_Audio_Text_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Gate-1 Voice + Text Intent Classification Notebook**

In [1]:
# Setup and Imports
# Install compatible versions
!pip -q install --upgrade --no-cache-dir "transformers==4.39.3" "huggingface_hub>=0.23" accelerate

# Imports (after pip cell finishes running)
import torch, numpy as np
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
from transformers import AutoTokenizer, AutoModel, pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 107.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 5.1.2 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.39.3 which is incompatible.


In [2]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Audio: Use Wav2Vec2 Feature Extractor instead of Processor
fe = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-large-xlsr-53")
xlsr = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-xlsr-53").to(device).eval()

# Text: BERT base
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert = AutoModel.from_pretrained("bert-base-uncased").to(device).eval()

# ASR: Whisper base
asr = pipeline("automatic-speech-recognition", model="openai/whisper-base", device=0 if torch.cuda.is_available() else -1)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json: 0.00B [00:00, ?B/s]

In [4]:
# Audio + Text Featurizer
import os, hashlib
import torch.nn.functional as F
import soundfile as sf
import torchaudio

TEXT_CACHE = "/content/features/text_embeds_cache"
os.makedirs(TEXT_CACHE, exist_ok=True)

# Util to generate a hash for each audio file (used as cache key)
def audio_hash(path):
    with open(path, "rb") as f:
        return hashlib.md5(f.read()).hexdigest()

# Use Whisper to get ASR transcript
def transcribe_whisper(path):
    return asr(path)["text"]

# Get text embedding using BERT
@torch.no_grad()
def bert_embed(text):
    tokens = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=64).to(device)
    h = bert(**tokens).last_hidden_state
    return F.adaptive_avg_pool1d(h.transpose(1,2), 1).squeeze().cpu().numpy()  # (768,)

@torch.no_grad()
def xlsr_embed(audio, sr=1600):
    inputs = fe(audio, sampling_rate=sr, return_tensors="pt").input_values.to(device)
    h = xlsr(inputs).last_hidden_state
    return F.adaptive_avg_pool1d(h.transpose(1,2), 1).squeeze().cpu().numpy()

def load_wav(path, sr=16000):
    y, s = sf.read(path)
    if s != sr:
        y = torchaudio.functional.resample(torch.tensor(y).float().unsqueeze(0), s, sr).squeeze(0).numpy()
    if y.ndim > 1: y = y.mean(-1)
    return y

# Complete featurizer: XLSR audio + BERT text
def featurize(path, sr=16000):
    audio = load_wav(path, sr).astype(np.float32)
    a_feat = xlsr_embed(audio, sr)  # (1024,)

    # Cache logic
    key = os.path.join(TEXT_CACHE, f"{audio_hash(path)}.npy")
    if os.path.exists(key):
        t_feat = np.load(key)
    else:
        text = transcribe_whisper(path)
        t_feat = bert_embed(text)
        np.save(key, t_feat)

    return np.concatenate([a_feat, t_feat])  # (1024 + 768 = 1792,)

In [5]:
# Download & Extract MELD Dataset
KAGGLE_USERNAME = "senudirupasinghe"
KAGGLE_KEY = "7780e1bc02634783fb08137fa45db94e"

!pip install kaggle --upgrade --quiet
!mkdir -p /root/.kaggle
!echo '{"username":"<KAGGLE_USERNAME>","key":"<KAGGLE_KEY>"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d zaber666/meld-dataset -p /content/ --unzip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.4/256.4 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 140.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 9.9 MB/s eta 0:00:00
Dataset URL: https://www.kaggle.com/datasets/zaber666/meld-dataset
License(s): CC0-1.0
100% 11.0G/11.0G [01:10<00:00, 266MB/s]
100% 11.0G/11.0G [01:10<00:00, 167MB/s]


In [6]:
# Extract CSV paths and audio mapping
import os, glob, pandas as pd
MELD_ROOT = "/content/MELD.Raw"
AUDIO_OUT = "/content/audio/meld"

# Index mp4 files
mp4_index = {}
for p in glob.glob(os.path.join(MELD_ROOT, "**", "*.mp4"), recursive=True):
    mp4_index[os.path.basename(p).lower()] = p

# Extract audio from MP4 to WAV
import subprocess, soundfile as sf
os.makedirs(AUDIO_OUT, exist_ok=True)
def mp4_to_wav(dialogue_id, utterance_id):
    mp4 = mp4_index.get(f"dia{dialogue_id}_utt{utterance_id}.mp4")
    if not mp4: return None
    out_path = os.path.join(AUDIO_OUT, f"{dialogue_id}_{utterance_id}.wav")
    if not os.path.exists(out_path):
        subprocess.run(["ffmpeg","-y","-i",mp4,"-vn","-ac","1","-ar","16000","-sample_fmt","s16",out_path],
                       stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return out_path if os.path.exists(out_path) else None


In [7]:
import glob
import subprocess

AUDIO_OUT = "/content/audio/meld"
os.makedirs(AUDIO_OUT, exist_ok=True)

# Build a fast lookup once
mp4_index = {}
for p in glob.glob("/content/MELD-RAW/MELD.Raw/**/*.mp4", recursive=True):
    mp4_index[os.path.basename(p).lower()] = p

def mp4_to_wav(dialogue_id, utterance_id):
    name = f"dia{dialogue_id}_utt{utterance_id}.mp4".lower()
    src = mp4_index.get(name)
    if not src:
        return None
    dst = os.path.join(AUDIO_OUT, f"{dialogue_id}_{utterance_id}.wav")
    if not os.path.exists(dst):
        subprocess.run([
            "ffmpeg", "-y", "-i", src, "-vn",
            "-ac", "1", "-ar", "16000", "-sample_fmt", "s16", dst
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    return dst if os.path.exists(dst) else None

In [8]:
# Generate Manifest CSVs
def build_manifest(csv_path, out_path, lang="en", domain="general"):
    df = pd.read_csv(csv_path)
    paths, labels = [], []
    emap = {
        "engaged":    {"neutral","joy","happy","excited"},
        "hesitating": {"sadness","fear","surprise"},
        "rejecting":  {"anger","disgust","contempt"}
    }
    label_map = {"engaged":0,"hesitating":1,"rejecting":2}
    for i, row in df.iterrows():
        emo = row["Emotion"].lower().strip()
        label = None
        for k,v in emap.items():
            if emo in v: label = label_map[k]
        if label is None: continue
        wav = mp4_to_wav(row["Dialogue_ID"], row["Utterance_ID"])
        if wav: paths.append(wav); labels.append(label)
    pd.DataFrame({
        "path": paths,
        "label": labels,
        "lang": lang,
        "domain": domain
    }).to_csv(out_path, index=False)

build_manifest("/content/MELD-RAW/MELD.Raw/train/train_sent_emo.csv", "/content/manifest_meld_train.csv")
build_manifest("/content/MELD-RAW/MELD.Raw/dev_sent_emo.csv", "/content/manifest_meld_dev.csv")
build_manifest("/content/MELD-RAW/MELD.Raw/test_sent_emo.csv", "/content/manifest_meld_test.csv")

In [9]:
# Rebuild combined feature arrays (audio + text)
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

def build_npz(manifest_csv, out_npz):
    man = pd.read_csv(manifest_csv)
    X, y = [], []
    for p, lab in tqdm(zip(man["path"], man["label"]), total=len(man), desc=f"Featurize {os.path.basename(manifest_csv)}"):
        try:
            x = featurize(p)
            X.append(x); y.append(int(lab))
        except Exception as e:
            print(f"[!] Failed: {p} -> {e}")
    X = np.stack(X).astype(np.float32)
    y = np.array(y, dtype=np.int64)
    np.savez(out_npz, X=X, y=y)
    print(out_npz, X.shape, y.shape)

# Trigger rebuilds (may take 10+ mins depending on GPU/CPU)
build_npz("/content/manifest_meld_train.csv", "/content/features/meld_train.npz")
build_npz("/content/manifest_meld_dev.csv", "/content/features/meld_dev.npz")
build_npz("/content/manifest_meld_test.csv", "/content/features/meld_test.npz")

Featurize manifest_meld_train.csv:   0%|          | 10/9988 [00:20<1:55:56,  1.43it/s]--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.12/logging/__init__.py", line 1160, in emit
    msg = self.format(record)
          ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/logging/__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/logging/__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/logging/__init__.py", line 392, in getMessage
    msg = msg % self.args
          ~~~~^~~~~~~~~~~
TypeError: not all arguments converted during string formatting
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File

/content/features/meld_train.npz (9988, 1792) (9988,)


Featurize manifest_meld_dev.csv: 100%|██████████| 1109/1109 [02:44<00:00,  6.75it/s]


/content/features/meld_dev.npz (1109, 1792) (1109,)


Featurize manifest_meld_test.csv: 100%|██████████| 2609/2609 [05:10<00:00,  8.40it/s]


/content/features/meld_test.npz (2609, 1792) (2609,)
